# 5.4 単純なレイヤの実装

## 5.4.1 乗算レイヤの実装

各ノードでの演算に対応するクラスを実装


- 乗算レイヤ

In [23]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy


- 乗算レイヤを用いて順伝播を実装

In [24]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("price:", int(price))


price: 220


- 逆伝播を実装

In [25]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

dApple: 2.2
dApple_num: 110
dTax: 200


## 5.4.2 加算レイヤの実装


- 加算レイヤ

In [26]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy


- 順伝播、逆伝播を実装

In [27]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  
orange_price = mul_orange_layer.forward(orange, orange_num) 
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  
price = mul_tax_layer.forward(all_price, tax)  

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


# 5.5 活性化関数レイヤの実装
計算グラフの考え方をニューラルネットワークに適用する  
ニューラルネットワークを構成する「層（レイヤ）」を一つのクラスとして実装（今回は活性化関数の実装のみ）

## 5.5.1 ReLU レイヤ

ReLu関数は以下の式で表される
$$
y = \left\{
\begin{array}{ll}
x & (x > 0)\\
0 & (x \leq 0)
\end{array}
\right.
$$

$$
\frac{\partial y}{\partial x} = \left\{
\begin{array}{ll}
1 & (x > 0)\\
0 & (x \leq 0)
\end{array}
\right.
$$


- ReLUレイヤの実装

In [28]:
import numpy as np

In [29]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


## 5.5.2 Sigmoidレイヤ

Sigmoid関数は以下の式で表される
$$
y = \frac{1}{1 + e^{-x}}
$$

$$\frac{\partial y}{\partial x}  = \frac{e^{-x}}{(1 + e^{-x})^2} = \frac{1}{1 + e^{-x}}\cdot \frac{e^{-x}}{1 + e^{-x}} = {y} \cdot (1 - {y})$$

- Sigmoidレイヤの実装

In [30]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx
